In [ ]:
############# Flask Modules Setup ##############

from flask import Flask, render_template, request, make_response, jsonify
from flask_login import LoginManager, login_user, logout_user, login_required, UserMixin

import MySQLdb
######### Initialize Flask App ##########
app = Flask(__name__)
######### MySQL Database Setup ##########
#connection = MySQLdb.connect(host="HallieZHU.mysql.pythonanywhere-services.com",
#                    user="HallieZHU",
#                    passwd="yukinoNaNa1997",
#                    db="HallieZHU$Assignment",
#                    charset='utf8mb4')
######### Login Manager Setup ##########
login_manager = LoginManager()
login_manager.init_app(app)
login_manager.login_view = 'login'
app.config['SECRET_KEY'] = "lkkajdghdadkglajkgah"

@login_manager.user_loader
def load_user(user_id):
    return User(user_id)

class User(UserMixin):
  def __init__(self,id):
    self.id = id

############ Web Page Routes Setup ###############

@app.route("/")
@login_required
def home():
    return render_template('home.html')

@app.route("/login")
def login():
    message = '请先登录！'
    return render_template('login.html', message=message)

@app.route('/query_auth')
def query_auth():
    username = request.args.get("username")
    password = request.args.get("password")
    if username == "hallieshyu@xx.com" and password == "0108":
        login_user(User(1))
        message = "你好，" + username + "欢迎来到寻书侦探网站。"
        return make_response(message, 200, )
    else:
        message = "抱歉，密码错误！"
        return make_response(message,401,{'WWW-Authenticate':'Basic realm="Login Required"'})

@app.route("/form_auth",methods=['POST'])
def form_auth():
    username = request.form['email']
    password = request.form['pwd']
    if username == "hallieshyu@xx.com" and password == "0108":
        login_user(User(1))
        message = "你好，" + username + "欢迎来到寻书侦探网站。"
        return render_template('home.html', message=message)
    else:
        message = "抱歉，密码错误！"
        return render_template('login.html',message=message)

@app.route('/album')
@login_required
def album():
    connection = MySQLdb.connect(host="HallieZHU.mysql.pythonanywhere-services.com",
                     user="HallieZHU",
                     passwd="yukinoNaNa1997",
                     db="HallieZHU$Assignment",
                     charset='utf8mb4')
    user = {"name":"Bernard"}
    with connection.cursor() as cursor:
        sql = "SELECT * FROM assign4"
        cursor.execute(sql)
        result = cursor.fetchall()
    final_result = [list(i) for i in result]
    dataset=[]
    dict={}
    for i in final_result:
        dict['name'] = i[0]
        dict['poster'] = i[1]
        dict['cast'] = i[2]
        dict['type'] = i[3]
        dict['date'] = i[4]
        dict['rating'] = i[5]
        dataset.append(dict.copy()) #markers.append(fld.copy())
    cursor.close()
    connection.close()
    return render_template('product2.html',album_user=user, dataset=dataset)

@app.route('/logout/')
@login_required
def logout():
    logout_user()
    message = '欢迎下次再来！'
    return render_template('login.html',message=message)

@app.errorhandler(500)
def internal_error(error):
    message = '抱歉，密码错误!'
    return render_template('login.html',message=message),500

######### API Endpoints ##########

@app.route('/api_auth', methods=['POST'])
def api_auth():
    username = request.json['username']
    password = request.json['password']
    if username == "hallieshyu@xx.com" and password == "0108":
        login_user(User(1))
        message = "你好，" + username + "欢迎来到寻书侦探网站。"
        return jsonify({'response':'ok!','message':message})
    else:
        message = '抱歉，密码错误！'
        return jsonify({'response':'Invalid!','message':'Cannot authenticate.'})

@app.route('/api_album', methods=['POST'])
@login_required
def api_album():
    connection = MySQLdb.connect(host="HallieZHU.mysql.pythonanywhere-services.com",
                     user="HallieZHU",
                     passwd="yukinoNaNa1997",
                     db="HallieZHU$Assignment",
                     charset='utf8mb4')
    with connection.cursor() as cursor:
        sql = "SELECT * FROM assign4"
        cursor.execute(sql)
        result = cursor.fetchall()
    final_result = [list(i) for i in result]
    dataset=[]
    dict={}
    for i in final_result:
        dict['name'] = i[0]
        dict['poster'] = i[1]
        dict['cast'] = i[2]
        dict['type'] = i[3]
        dict['date'] = float(i[4])
        dict['rating'] = i[5]
        dataset.append(dict.copy()) #markers.append(fld.copy())
    cursor.close()
    connection.close()
    return jsonify({'Album': dataset})

######### Run Flask Web App at Port 9030 ##########

if __name__ == '__main__':
    app.run(debug = True)